In [16]:
# # install chromium, its driver, and selenium
# !apt-get update
# !apt install chromium-chromedriver
# !cp /usr/lib/chromium-browser/chromedriver /usr/bin
# !pip install selenium

"Montiamo" la cartella Drive in cui sono salvati i vari moduli che serviranno per eseguire il codice. In particolare la cartella `modules` che contiene i files:
* `dataset.py`: contiene la classe `StoreDataset()` e le funzioni utili per la gestione del dataset stesso.
* `regressor.py`: contiene la classe `Regressor()` che, banalmente richiama `XGBRegressor()`

> Nota: la classe `Regressor()` potrebbe essere migliorata aggiungendo altri regressori, come ad esempio un regressore lineare.


In [17]:
from google.colab import drive
drive.mount('/content/drive')

import sys, os
base_path = '/content/drive'
path = 'MyDrive/university/msc/pg_businesscase/modules'

if os.path.join(base_path, path) not in sys.path:
    sys.path.append(os.path.join(base_path, path))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Importiamo i moduli necessari.

In [18]:
import dataset as dt
import regressor as rg
import utils
import pandas as pd

Creiamo le instanze di classe `StoreDataset()` e `Regressor()`.
In particolare per il regressore dobbiamo definire il tipo (`xgb`) e il numero di stimatori utilizzati.

In [19]:
data = dt.StoreDataset('/content/drive/MyDrive/university/msc/pg_businesscase/data/definitivo.csv')
regressor = rg.Regressor(type='xgb', n_estimators=600)

In [20]:
data.initial

,precip_luglio,temp_max_luglio,n_hotels,n_camping,n_b&b,total_accomodations,latitude,longitude,distanza_mare,regione,provincia,comune,popolazione,superficie,categoria_comune,indice_d,indice_p,indice_t,indice_sintesi,sustainability_score,tot_pop,maschi,femmine,celibi/nubili,coniugati,divorziati,single,tot_famiglie,componenti_famiglie,eta_minore_5,eta_5/9,eta_10/14,eta_15/19,eta_20/24,eta_25/29,eta_30/34,eta_35/39,eta_40/44,eta_45/49,eta_50/54,eta_55/59,eta_60/64,eta_65/69,eta_70/74,eta_maggiore_74,con_laurea,con_diploma,con_media,con_elementare,occupati,disoccupati,studenti,income,vendite
0,39,246,205,14,3,222,44.404043,8.677308,0.997326,LIGURIA,Genova,Arenzano,11416.0,24.30,C,4.0,5.0,4.0,5.0,56.019,237741,0.470537,0.529463,0.369423,0.457060,0.035715,0.005371,115512,2.047095,0.036973,0.037827,0.038197,0.038921,0.040182,0.040906,0.050059,0.068486,0.078750,0.081122,0.071292,0.064915,0.070505,0.064553,0.070345,0.146967,0.103533,0.308941,0.284755,0.196807,0.387565,0.025271,0.050837,22447.48668,8110.053471
1,39,246,195,10,3,208,44.405538,8.678307,0.839883,LIGURIA,Genova,Arenzano,11416.0,24.30,C,4.0,5.0,4.0,5.0,56.019,184274,0.470235,0.529765,0.367930,0.459479,0.035350,0.006132,89177,2.053713,0.036185,0.037347,0.038041,0.039018,0.039897,0.040288,0.048260,0.067280,0.078041,0.080576,0.070949,0.065669,0.072045,0.065761,0.071915,0.148730,0.106727,0.312871,0.281261,0.196023,0.384683,0.024626,0.051646,22687.91364,7313.202333
2,39,246,186,8,3,197,44.408278,8.683755,0.673746,LIGURIA,Genova,Arenzano,11416.0,24.30,C,4.0,5.0,4.0,5.0,56.019,149890,0.469624,0.530376,0.366229,0.460084,0.035266,0.006832,72704,2.047563,0.035506,0.037314,0.037774,0.038655,0.039696,0.039749,0.047695,0.066362,0.077217,0.079859,0.071419,0.065995,0.073260,0.066128,0.072813,0.150557,0.108480,0.313897,0.280172,0.195143,0.382974,0.024358,0.051278,22851.80165,6916.525084
3,38,238,230,14,3,247,44.399782,8.664989,0.949236,LIGURIA,Genova,Arenzano,11416.0,24.30,C,4.0,5.0,4.0,5.0,56.019,342006,0.471381,0.528619,0.374996,0.451817,0.036210,0.006664,166090,2.045439,0.037619,0.038403,0.038350,0.039915,0.041105,0.042543,0.051289,0.069025,0.078241,0.081461,0.071569,0.064929,0.069663,0.063180,0.068493,0.144217,0.103352,0.306609,0.285475,0.194473,0.389654,0.026488,0.051318,21709.29064,8071.015278
4,39,246,197,10,3,210,44.403391,8.684478,0.297494,LIGURIA,Genova,Arenzano,11416.0,24.30,C,4.0,5.0,4.0,5.0,56.019,193816,0.470369,0.529631,0.367808,0.459962,0.035173,0.005856,93736,2.055571,0.036241,0.037484,0.038371,0.039140,0.039620,0.040198,0.048484,0.067585,0.078255,0.080809,0.071047,0.065634,0.071810,0.065547,0.071650,0.148125,0.105714,0.312038,0.281803,0.196872,0.385216,0.024549,0.051539,22657.00105,7324.050116
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,26,285,10,0,0,10,40.899262,16.843970,24.689584,PUGLIA,Bari,Acquaviva delle Fonti,20523.0,132.02,P,1.0,4.0,4.0,3.0,41.610,28371,0.487152,0.512848,0.403264,0.503155,0.012337,0.003489,10588,2.670193,0.044130,0.043989,0.049276,0.056149,0.060026,0.064432,0.065525,0.073314,0.072010,0.076980,0.074054,0.066406,0.063621,0.045680,0.045927,0.098481,0.104332,0.284516,0.265024,0.200169,0.344225,0.042191,0.081351,13194.03581,0.000000
476,23,288,47,1,2,50,41.116401,16.874224,0.948982,PUGLIA,Bari,Bari,320862.0,117.41,A,5.0,5.0,4.0,5.0,41.610,380507,0.479347,0.520653,0.402544,0.488490,0.018728,0.003826,148178,2.558079,0.041460,0.044937,0.047353,0.051082,0.054827,0.055636,0.061528,0.074779,0.078004,0.080837,0.073350,0.065754,0.066472,0.056304,0.050446,0.097231,0.139503,0.284318,0.256526,0.183171,0.342958,0.038081,0.069930,17779.43641,0.000000
477,3,281,19,5,5,29,37.304345,13.592907,3.842853,SICILIA,Agrigento,Agrigento,58956.0,245.32,G,5.0,5.0,4.0,4.0,54.425,43024,0.479941,0.520059,0.418208,0.473247,0.014550,0.002929,16790,2.554973,0.042930,0.042744,0.051646,0.058688,0.066103,0.0589

Chiamiamo il metodo `StoreDataset.get_dataset()` che ritorna i tre dataset che verrano utilizzati per il training del regressore (`X_train`, `Y_train`) e per la predizione stessa (`test`).

In [21]:
X_train, Y_train, test = data.get_datasets()

In [22]:
X_train

,precip_luglio,temp_max_luglio,n_hotels,n_camping,total_accomodations,latitude,longitude,distanza_mare,sustainability_score,maschi,femmine,celibi/nubili,coniugati,divorziati,eta_minore_5,eta_5/9,eta_10/14,eta_35/39,eta_40/44,eta_45/49,eta_55/59,eta_60/64,eta_65/69,eta_70/74,eta_maggiore_74,con_laurea,con_diploma,con_media,con_elementare,occupati,disoccupati,studenti,income
0,39,246,205,14,222,44.404043,8.677308,0.997326,56.019,0.470537,0.529463,0.369423,0.457060,0.035715,0.036973,0.037827,0.038197,0.068486,0.078750,0.081122,0.064915,0.070505,0.064553,0.070345,0.146967,0.103533,0.308941,0.284755,0.196807,0.387565,0.025271,0.050837,22447.486680
1,39,246,195,10,208,44.405538,8.678307,0.839883,56.019,0.470235,0.529765,0.367930,0.459479,0.035350,0.036185,0.037347,0.038041,0.067280,0.078041,0.080576,0.065669,0.072045,0.065761,0.071915,0.148730,0.106727,0.312871,0.281261,0.196023,0.384683,0.024626,0.051646,22687.913640
2,39,246,186,8,197,44.408278,8.683755,0.673746,56.019,0.469624,0.530376,0.366229,0.460084,0.035266,0.035506,0.037314,0.037774,0.066362,0.077217,0.079859,0.065995,0.073260,0.066128,0.072813,0.150557,0.108480,0.313897,0.280172,0.195143,0.382974,0.024358,0.051278,22851.801650
3,38,238,230,14,247,44.399782,8.664989,0.949236,56.019,0.471381,0.528619,0.374996,0.451817,0.036210,0.037619,0.038403,0.038350,0.069025,0.078241,0.081461,0.064929,0.069663,0.063180,0.068493,0.144217,0.103352,0.306609,0.285475,0.194473,0.389654,0.026488,0.051318,21709.290640
4,39,246,197,10,210,44.403391,8.684478,0.297494,56.019,0.470369,0.529631,0.367808,0.459962,0.035173,0.036241,0.037484,0.038371,0.067585,0.078255,0.080809,0.065634,0.071810,0.065547,0.071650,0.148125,0.105714,0.312038,0.281803,0.196872,0.385216,0.024549,0.051539,22657.001050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2,271,3,1,4,37.101036,13.941010,0.394972,54.425,0.486634,0.513366,0.416699,0.490325,0.009033,0.044537,0.055534,0.061530,0.072317,0.078548,0.073050,0.057209,0.056555,0.047679,0.045846,0.090619,0.061032,0.209620,0.312335,0.208939,0.253554,0.034666,0.073652,9371.646951
236,9,292,43,4,58,37.561966,15.129172,2.019643,27.484,0.476024,0.523976,0.427504,0.458979,0.020371,0.045759,0.048822,0.050462,0.074451,0.077307,0.077848,0.065432,0.062626,0.049598,0.045766,0.089568,0.137586,0.311010,0.262757,0.151892,0.323062,0.049346,0.077455,15327.354980
237,9,292,43,6,73,37.549500,15.049017,6.550932,27.484,0.480070,0.519930,0.436466,0.455931,0.018462,0.049711,0.051869,0.052642,0.075496,0.076363,0.075931,0.063533,0.059930,0.047257,0.042756,0.082801,0.108225,0.281437,0.292399,0.167300,0.303579,0.052117,0.071445,14344.312540
238,92,248,203,11,214,46.489574,11.332100,138.080318,75.865,0.482846,0.517154,0.456958,0.412495,0.034209,0.050829,0.051371,0.051272,0.074777,0.084035,0.083997,0.057829,0.057079,0.057106,0.052384,0.096853,0.113193,0.334535,0.269108,0.161507,0.473802,0.016034,0.060661,26581.195250


Alleniamo il regressore e, successivamente, facciamo le predizioni.

In [23]:
regressor.fit(X_train, Y_train)
results = regressor.predict(test)

Quando le predizioni sono disponibili, ovvero - in questo caso - le vendite dei negozi sono state predette, aggiorniamo il dataset con questi risultati chiamando il metodo `StoreDataset.update_with_results()`. Se `sort_by_sales` è `True`, il dataset viene ordinato in ordine decrescente in base alle vendite dei singoli negozi.

In [24]:
data.update_with_results(results=results, sort_by_sales=True)

Con il dataset aggiornato (che sarà chiamato `data.final`) possiamo selezionare solo le features che riteniamo "più importanti" ai fini del clustering.

Per fare questo ci avvaliamo del metodo `StoreDataset.correlated_features()` che prende come parametri di input il dataset su cui fare la correlazione e la colonna di riferimento (per noi "vendite"). La selezione delle features più correlate con le "vendite" avviene applicando un threshold al valore assoluto della correlazione tra una features e "vendite". Quelle che superano il threshold sono selezionate e inserite nella lista `cols_to_keep`.

> Nota: la colonna vendite non viene selezionata per il clustering.

Il metodo `StoreDataset.prepare_for_clustering()` "taglia" il dataset passato come parametro considerando solo le colonne da mantenere (o eventualmente si potrebbero inserire quelle da tagliare con il parametro `cols_to_drop`, non presente in questo esempio).

In [25]:
cols_to_keep = data.correlated_features(dataframe=data.train_set, reference_feature='vendite', threshold=0.2)
cols_to_keep
data_clustering = dt.prepare_for_clustering(data.final, cols_to_keep=cols_to_keep[:-1])

In [26]:
cols_to_keep

Index(['precip_luglio', 'temp_max_luglio', 'n_camping', 'latitude',
       'longitude', 'distanza_mare', 'maschi', 'femmine', 'celibi/nubili',
       'coniugati', 'eta_minore_5', 'eta_5/9', 'eta_10/14', 'eta_40/44',
       'eta_45/49', 'eta_55/59', 'eta_60/64', 'eta_maggiore_74', 'con_laurea',
       'con_diploma', 'con_media', 'con_elementare', 'occupati', 'disoccupati',
       'studenti', 'vendite'],
      dtype='object')

Quando il dataset è pronto, eseguiamo il KMeans clustering con un certo numero di cluster (da decidere).

In [27]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3).fit(data_clustering)

Aggiorniamo il dataset con i labels ottenuti dal clustering, e otteniamo `data.clusterized`.

Per questo ci avvaliamo del metodo `StoreDataset.update_with_clusters()` che prende in input i labels appena creati.

In [28]:
data.update_with_clusters(labels=kmeans.labels_)

A questo punto possiamo plottare (in base a latitudine e longitudine) i clusters ottenuti, con il metodo `StoreDataset.plot_clusters_ll()`.

In [29]:
clusters = data.clusterized.groupby('cluster').mean()
clusters.to_csv('/content/drive/MyDrive/university/msc/pg_businesscase/clusters_table.csv')
clusters.to_excel('/content/drive/MyDrive/university/msc/pg_businesscase/clusters_table.xlsx')
clusters

,precip_luglio,temp_max_luglio,n_hotels,n_camping,n_b&b,total_accomodations,latitude,longitude,distanza_mare,popolazione,indice_d,indice_p,indice_t,indice_sintesi,sustainability_score,tot_pop,maschi,femmine,celibi/nubili,coniugati,divorziati,single,tot_famiglie,componenti_famiglie,eta_minore_5,eta_5/9,eta_10/14,eta_15/19,eta_20/24,eta_25/29,eta_30/34,eta_35/39,eta_40/44,eta_45/49,eta_50/54,eta_55/59,eta_60/64,eta_65/69,eta_70/74,eta_maggiore_74,con_laurea,con_diploma,con_media,con_elementare,occupati,disoccupati,studenti,income,vendite
cluster,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,31.743083,277.845850,31.731225,2.229249,1.814229,35.774704,41.735815,13.408883,15.516476,67224.707510,3.502146,3.871245,3.467811,3.527897,51.693024,160232.727273,0.484838,0.515162,0.408744,0.472969,0.020097,0.004128,64819.320158,2.443029,0.043842,0.045207,0.046302,0.048925,0.053271,0.056168,0.063745,0.075964,0.079345,0.079934,0.070394,0.064465,0.064050,0.052119,0.052102,0.104167,0.107320,0.291553,0.274937,0.183514,0.365858,0.038730,0.068185,16328.059319,5624.307764
1,99.325000,219.462500,74.337500,4.587500,1.012500,79.937500,45.999292,9.672658,153.901287,28413.637500,4.257143,4.485714,4.100000,4.357143,64.007537,177867.650000,0.487603,0.512397,0.435555,0.434520,0.030417,0.007986,75054.000000,2.285509,0.050079,0.049812,0.049343,0.047282,0.048206,0.052910,0.061635,0.077115,0.084157,0.083991,0.070327,0.061388,0.061236,0.053626,0.052145,0.096747,0.095465,0.293133,0.296291,0.192608,0.454009,0.020276,0.056665,21537.908288,3045.571072
2,44.789116,271.897959,48.761905,1.428571,2.945578,53.136054,43.226977,12.333032,72.393565,54340.707483,3.500000,3.781690,3.647887,3.500000,54.875490,149605.836735,0.482275,0.517725,0.399796,0.471243,0.022990,0.005151,65983.006803,2.373411,0.043393,0.043532,0.043614,0.045532,0.049372,0.055031,0.063280,0.075149,0.079101,0.079514,0.070828,0.063899,0.064650,0.051809,0.053866,0.117430,0.116879,0.299882,0.261127,0.188931,0.405508,0.031225,0.064478,18468.564545,4337.796458


In [30]:
map = utils.plot_map_clusters(data.clusterized)
map.save('/content/drive/MyDrive/university/msc/pg_businesscase/map.html')
map